In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import wrds
import datetime

In [3]:
conn = wrds.Connection()

# set sample date range
begdate = '01/01/1980'
enddate = '10/31/2023'

# set CRSP date range a bit wider to guarantee collecting all information
crsp_begdate = '01/01/1979'
crsp_enddate = '11/20/2023'

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [36]:
#################################
# Step 0: Read in ICLINK output #
#################################

# iclink.csv is the output from the python program iclink
# it contains the linking between crsp and ibes
# read iclink
iclink = pd.read_csv('iclink.csv')

In [37]:
##################################
# Step 1. S&P 500 Index Universe #
##################################

# All companies that were ever included in S&P 500 index as an example
# Old version of the code uses comp.idxcst_his

# New code uses crsp.msp500list
# Linking Compustat GVKEY and IBES Tickers using ICLINK               
# For unmatched GVKEYs, use header IBTIC link in Compustat Security file 


sp500 = conn.raw_sql(f"""
                        select a.*
                        from crsp.msp500list as a;
                        """, date_cols=['start', 'ending'])

# CCM data
_ccm = conn.raw_sql(""" select gvkey, lpermco as permco, lpermno as permno, 
                        linkdt, linkenddt 
                        from crsp.ccmxpf_linktable 
                        where usedflag=1 
                        and linkprim in ('P', 'C')""", date_cols=['linkdt', 'linkenddt'])

_ccm[['permco', 'permno']] = _ccm[['permco', 'permno']].astype(int)

# Fill linkenddt missing value (.E in SAS dataset) with today's date
_ccm['linkenddt'] = _ccm.linkenddt.fillna(datetime.date.today())

_sec = conn.raw_sql(""" select ibtic, gvkey from comp.security """)


# Start the sequence of left join
gvkey = pd.merge(sp500, _ccm, how='left', on=['permno'])
gvkey = pd.merge(gvkey, _sec.loc[_sec.ibtic.notna()], how='left', on=['gvkey'])

# high quality links from iclink
# score = 0 or 1
iclink_hq = iclink.loc[(iclink.score <=1)]

gvkey = pd.merge(gvkey, iclink_hq, how='left', on=['permno'])

# fill missing ticker with ibtic
gvkey.ticker = np.where(gvkey.ticker.notnull(), gvkey.ticker, gvkey.ibtic)

# Keep relevant columns and drop duplicates if there is any
gvkey = gvkey[['gvkey', 'permco', 'permno', 'linkdt', 'linkenddt','ticker']]

gvkey = gvkey.drop_duplicates()

# date ranges from gvkey
# min linkdt for ticker and permno combination
gvkey_mindt = gvkey.groupby(['ticker','permno']).linkdt.min().reset_index()

# max linkenddt for ticker and permno combination
gvkey_maxdt = gvkey.groupby(['ticker','permno']).linkenddt.max().reset_index()

# link date range 
gvkey_dt = pd.merge(gvkey_mindt, gvkey_maxdt, how='inner', on=['ticker','permno'])


# null unused dataframes to free memory
sp500 = None
_ccm = None
_sec = None
iclink = None
gvkey_mindt = None
gvkey_maxdt = None

#######################################
# Only gvkey is left                  #
#######################################

/Users/benkan45d6/opt/anaconda3/lib/python3.9/site-packages/pandas/core/algorithms.py:560: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  uniques, codes = table.factorize(


In [7]:
#######################################
# Step 2. Extract Estimates from IBES #
#######################################

# Extract estimates from IBES Unadjusted file and select    
# the latest estimate for a firm within broker-analyst group
# "fpi in (6,7)" selects quarterly forecast for the current 
# and the next fiscal quarter    

ibes_temp = conn.raw_sql(f"""
                        select ticker, estimator, analys, pdf, fpi, value, 
                        fpedats, revdats, revtims, anndats, anntims
                        from ibes.detu_epsus 
                        where fpedats between '{begdate}' and '{enddate}'
                        and (fpi='6' or fpi='7')
                        """, date_cols = ['revdats', 'anndats', 'fpedats'])


# merge to get date range linkdt and linkenddt to fulfill date requirement
ibes_temp = pd.merge(ibes_temp, gvkey_dt, how='left', on=['ticker'])
ibes_temp = ibes_temp.loc[(ibes_temp.linkdt<=ibes_temp.anndats) & (ibes_temp.anndats <= ibes_temp.linkenddt)]

In [8]:
# Count number of estimates reported on primary/diluted basis 

p_sub = ibes_temp[['ticker','fpedats','pdf']].loc[ibes_temp.pdf=='P']
d_sub = ibes_temp[['ticker','fpedats','pdf']].loc[ibes_temp.pdf=='D']

p_count = p_sub.groupby(['ticker','fpedats']).pdf.count().reset_index().rename(columns={'pdf':'p_count'})
d_count = d_sub.groupby(['ticker','fpedats']).pdf.count().reset_index().rename(columns={'pdf':'d_count'})

ibes = pd.merge(ibes_temp, d_count, how = 'left', on=['ticker', 'fpedats'])
ibes = pd.merge(ibes, p_count, how='left', on =['ticker','fpedats'])
ibes['d_count'] = ibes.d_count.fillna(0)
ibes['p_count'] = ibes.p_count.fillna(0)

# Determine whether most analysts report estimates on primary/diluted basis
# following Livnat and Mendenhall (2006)                                   

ibes['basis']=np.where(ibes.p_count>ibes.d_count, 'P', 'D')

ibes = ibes.sort_values(by=['ticker','fpedats','estimator','analys','anndats', 'anntims', 'revdats', 'revtims'])\
.drop(['linkdt', 'linkenddt','p_count','d_count', 'pdf', 'fpi'], axis=1)

# Keep the latest observation for a given analyst
# Group by company fpedats estimator analys then pick the last record in the group

ibes_1 = ibes.groupby(['ticker','fpedats','estimator','analys']).apply(lambda x: x.index[-1]).to_frame().reset_index()

# reset index to the old dataframe index for join in the next step
ibes_1=ibes_1.set_index(0)

# Inner join with the last analyst record per group
ibes = pd.merge(ibes, ibes_1[['analys']], left_index=True, right_index=True)

# drop duplicate column
ibes=ibes.drop(['analys_y'], axis=1).rename(columns={'analys_x': 'analys'})


# null unneeded dataframes
ibes_temp = None
p_sub = None
d_sub = None
p_count = None
d_count = None
gvkey_dt = None

In [9]:
#######################################
# Step 3. Link Estimates with Actuals #
#######################################

# Link Unadjusted estimates with Unadjusted actuals and CRSP permnos  
# Keep only the estimates issued within 90 days before the report date

# Getting actual piece of data
ibes_act = conn.raw_sql(f"""
                        select ticker, anndats as repdats, value as act, pends as fpedats, pdicity
                        from ibes.actu_epsus 
                        where pends between '{begdate}' and '{enddate}'
                        and pdicity='QTR'
                        """, date_cols = ['repdats', 'fpedats'])


# Join with the estimate piece of the data

ibes1 = pd.merge(ibes, ibes_act, how='left', on = ['ticker','fpedats'])
ibes1['dgap'] = ibes1.repdats - ibes1.anndats

ibes1['flag'] = np.where( (ibes1.dgap>=datetime.timedelta(days=0)) & (ibes1.dgap<=datetime.timedelta(days=90)) & (ibes1.repdats.notna()) & (ibes1.anndats.notna()), 1, 0)

ibes1 = ibes1.loc[ibes1.flag==1].drop(['flag', 'dgap', 'pdicity'], axis=1)

# Select all relevant combinations of Permnos and Date

ibes1_dt1 = ibes1[['permno', 'anndats']].drop_duplicates()

ibes1_dt2 = ibes1[['permno', 'repdats']].drop_duplicates().rename(columns={'repdats':'anndats'})

ibes_anndats = pd.concat([ibes1_dt1, ibes1_dt2]).drop_duplicates()

# null dataframes no longer needed to free memory
ibes1_dt1 = None
ibes1_dt2 = None
ibes_act = None

In [10]:
# Adjust all estimate and earnings announcement dates to the closest
# preceding trading date in CRSP to ensure that adjustment factors won't
# be missing after the merge  

# unique anndats from ibes
uniq_anndats = ibes_anndats[['anndats']].drop_duplicates()

# unique trade dates from crsp.dsi
crsp_dats = conn.raw_sql(""" 
                            select date 
                            from crsp.dsi 
                         """, date_cols=['date'])

# Create up to 5 days prior dates relative to anndats

for i in range(0, 5):
    uniq_anndats[i] = uniq_anndats.anndats - datetime.timedelta(days=i)

# reshape (transpose) the df for later join with crsp trading dates

expand_anndats = uniq_anndats.set_index('anndats').stack().reset_index().\
rename(columns={'level_1':'prior', 0:'prior_date'})

# merge with crsp trading dates
tradedates = pd.merge(expand_anndats, crsp_dats, how='left', left_on=['prior_date'], right_on=['date'])

# create the dgap (days gap) variable for min selection
tradedates['dgap'] = tradedates.anndats-tradedates.date

# choosing the row with the smallest dgap for a given anndats
def minDgap(g):
    try:
        return g['dgap'].idxmin()
    except:
        return
tradedates = tradedates.loc[tradedates.groupby('anndats').apply(minDgap).dropna()]

tradedates = tradedates[['anndats', 'date']]

# null unneeded dataframes
unique_anndats = None

# Get SP500 est.

In [59]:
group = gvkey.groupby(['gvkey','permco','permno','ticker'])
def get_date(g):
    drange = pd.date_range('1800-01-01','1800-01-01') # dummy
    for r in range(len(g)):
        linkdt = g['linkdt'].iloc[r]
        linkenddt = g['linkenddt'].iloc[r]
        drange = drange.union(pd.date_range(linkdt,linkenddt))
    idx = g.reindex(drange).reset_index()['index']
    return idx
gvkey_withDate = gvkey.groupby(['gvkey','permco','permno','ticker']).apply(get_date).reset_index()
gvkey_withDate = gvkey_withDate.drop('level_4',axis=1)
gvkey_withDate = gvkey_withDate.rename(columns={'index':'date'})
gvkey_withDate

,gvkey,permco,permno,ticker,date
0,001013,2902.0,50906.0,ADCT,1800-01-01
1,001013,2902.0,50906.0,ADCT,1979-03-16
2,001013,2902.0,50906.0,ADCT,1979-03-17
3,001013,2902.0,50906.0,ADCT,1979-03-18
4,001013,2902.0,50906.0,ADCT,1979-03-19
...,...,...,...,...,...
20018406,326688,56329.0,17676.0,NVT,2023-11-16
20018407,326688,56329.0,17676.0,NVT,2023-11-17
20018408,326688,56329.0,17676.0,NVT,2023-11-18
20018409,326688,56329.0,17676.0,NVT,2023-11-19


In [61]:
earningsEst = pd.merge(gvkey_withDate,ibes1,how='inner',left_on=['ticker','date','permno'],right_on=['ticker','anndats','permno'])

In [62]:
earningsEst_d = earningsEst[earningsEst['basis']=='D']

In [63]:
earningsEst_d['sue'] = (earningsEst_d['act'] - earningsEst_d['value'])

/var/folders/vh/jb52psn93fb96vlsr5_kn4m80000gn/T/ipykernel_14317/1947643251.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  earningsEst_d['sue'] = (earningsEst_d['act'] - earningsEst_d['value'])


In [67]:
earningsEst_d

,gvkey,permco,permno,ticker,date,estimator,analys,value,fpedats,revdats,revtims,anndats,anntims,basis,repdats,act,sue
70,001013,2902.0,50906.0,ADCT,1993-09-30,942.0,129.0,0.39,1993-10-31,1995-01-26,15:00:07,1993-09-30,10:16:19,D,1993-12-17,0.37,-0.02
71,001013,2902.0,50906.0,ADCT,1993-10-13,183.0,995.0,0.37,1993-10-31,1995-01-26,11:37:10,1993-10-13,14:49:37,D,1993-12-17,0.37,0.00
72,001013,2902.0,50906.0,ADCT,1993-11-01,76.0,836.0,0.37,1993-10-31,1994-09-27,14:26:55,1993-11-01,13:03:01,D,1993-12-17,0.37,0.00
170,001013,2902.0,50906.0,ADCT,1998-01-12,114.0,20316.0,0.27,1998-01-31,1998-01-12,14:13:41,1998-01-12,11:07:14,D,1998-02-18,0.19,-0.08
171,001013,2902.0,50906.0,ADCT,1998-01-26,28.0,20575.0,0.18,1998-01-31,1998-08-05,15:58:36,1998-01-26,15:29:01,D,1998-02-18,0.19,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979231,326688,56329.0,17676.0,NVT,2022-07-31,1267.0,56596.0,0.59,2022-09-30,2022-10-05,16:32:25,2022-07-31,16:47:00,D,2022-10-28,0.66,0.07
979232,326688,56329.0,17676.0,NVT,2022-08-01,449.0,114868.0,0.58,2022-09-30,2022-10-04,08:00:39,2022-08-01,15:33:00,D,2022-10-28,0.66,0.08
979233,326688,56329.0,17676.0,NVT,2022-10-06,3051.0,74756.0,0.60,2022-09-30,2022-10-06,10:43:53,2022-10-06,01:34:00,D,2022-10-28,0.66,0.06
979234,326688,56329.0,17676.0,NVT,2022-10-07,4201.0,193800.0,0.60,2022-09-30,2022-10-19,02:47:47,2022-10-07,13:51:00,D,2022-10-28,0.66,0.06


In [82]:
def get_estSummary(g):
    g['est_mean'] = g['value'].mean()
    g['est_median'] = g['value'].median()
    g['est_std'] = g['value'].std()
    g['est_count'] = g['value'].count()
    return g[['permno','ticker','fpedats','repdats','act','est_mean','est_median','est_std','est_count']].iloc[0]
earningsEstSummary = earningsEst_d.groupby(['permno','ticker','fpedats','repdats']).apply(get_estSummary)

In [83]:
earningsEstSummary['sup_mean'] = earningsEstSummary['act'] - earningsEstSummary['est_mean']
earningsEstSummary['sup_median'] = earningsEstSummary['act'] - earningsEstSummary['est_median']

In [86]:
earningsEstSummary.reset_index(drop=True,inplace=True)

In [85]:
earningsEstSummary.to_csv('earningsEstSummary.csv')